In [3]:
"""
This script is suposed to produce plots of the results from
macroscopic approximation of the network-based microscopic model of divestment.
"""

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

class plot_routines(object):
    """
    this contains the routines to make different plots to compare the two
    types of model description
    """

    def __init__(self, base_path):
        self.micro_mean = np.load(base_path + 'micro/b_c_scan_micro_trajectory_trajectory')\
            [['mean_trajectory']].unstack(level=-1)
        self.micro_sem = np.load(base_path + 'micro/b_c_scan_micro_trajectory_trajectory')\
            [['sem_trajectory']].unstack(level=-1)
        self.macro_mean = np.load(base_path + 'macro/b_c_scan_macro_trajectory_trajectory')\
            [['mean_trajectory']].unstack(level=-1)
        self.macro_sem = np.load(base_path + 'macro/b_c_scan_macro_trajectory_trajectory')\
            [['sem_trajectory']].unstack(level=-1)

        self.data_sets = [self.micro_mean, self.micro_sem,
                     self.macro_mean, self.macro_sem]

        for d in self.data_sets:
            d.columns = d.columns.droplevel()

        self.variable_combos = [['x', 'y', 'z'], ['c', 'g'], ['mu_c^c', 'mu_d^d'],
                                ['mu_c^d', 'mu_d^c']]
        self.latex_labels = [['$x$', '$y$', '$z$'], ['$c$', '$g$'],
                             ['$\mu^{(c)}_c$', '$\mu^{(d)}_d$'],
                             ['$\mu^{(c)}_d$', '$\mu^{(d)}_c$']]

        self.colors = 'rgbk'

    def mk_plots(self, bd):

        # select data for given value of bd
        local_datasets = []
        for d in self.data_sets:
            local_datasets.append(d.xs(bd, level=0))

        phi_vals = local_datasets[0].index.levels[0].values

        l_phi = len(phi_vals)
        l_vars = len(self.variable_combos)

        fig = plt.figure()
        axes = [fig.add_subplot(l_vars, l_phi, i + 1 + j * l_phi)
                for i in range(l_phi)
                for j in range(l_vars)]

        for i, phi in enumerate(phi_vals):
            for j, variables in enumerate(self.variable_combos):
                ax_id = self.grid_index(i, j, l_phi, l_vars) - 1
                # local data set for specifiv value of phi
                ldp = []
                for d in local_datasets:
                    ldp.append(d.xs(phi, level=0))
                print(self.grid_index(i, j, l_phi, l_vars))
                ldp[0][variables]\
                    .plot(
                    ax=axes[ax_id],
                    legend=False,
                    color=self.colors)
                for k, variable in enumerate(variables):
                    upper_limit = np.transpose(ldp[0][[variable]].values \
                                  + ldp[1][[variable]].values)[0]
                    lower_limit = np.transpose(ldp[0][[variable]].values \
                                  - ldp[1][[variable]].values)[0]
                    axes[ax_id].fill_between(ldp[0].index.values,
                                     upper_limit, lower_limit,
                                     color=self.colors[k],
                                     alpha=0.2)
                    ldp[2][variables] \
                        .plot(
                        ax=axes[ax_id],
                        legend=False,
                        color=self.colors,
                        style='-.')

        return fig

    def mk_switchplots(self, bd, phi):
                # select data for given value of bd and phi
        local_datasets = []
        for d in self.data_sets:
            local_datasets.append(d.xs([bd, phi], level=[0, 1]))
        switchlist = np.load('switch_bd={}_phi={}.pkl'.format(bd, phi))

        fig = plt.figure()

        vars = self.variable_combos[2] + self.variable_combos[3]
        for v, var in enumerate(vars):
            direction = 1 if var[-2] == 'c' else 0
            capital_kind = '$K^{(c)}$' if var[-1] == 'c' else '$K^{(d)}$'
            print( var, capital_kind, direction)
            c1 = 'b'
            ax1 = fig.add_subplot(2, 2, v + 1)
            mu = local_datasets[0][[var]]
            emu = local_datasets[1][[var]]
            upper = np.transpose(np.array(mu.values + 2*emu.values))[0]
            lower = np.transpose(np.array(mu.values - 2*emu.values))[0]
            mu.plot(ax=ax1)
            ax1.plot(emu.index.values, upper, color=c1, alpha=0.4)
            ax1.plot(emu.index.values, lower, color=c1, alpha=0.4)
            plt.fill_between(emu.index.values,
                             upper,
                             lower,
                             color='k', alpha=0.2)
            l = len(switchlist[['direction']].values)
            ldots = 1000
            ts = []
            ks = []
            for i in range(ldots):
                k = np.random.randint(l)
                t = 1 if switchlist[['direction']].values[k] == direction else 0
                if t:
                    ts.append(switchlist.index.values[k])
                    ks.append(switchlist[[capital_kind]].values[k])
            ax1.plot(ts, ks, 'o'+c1, alpha=0.2, label=capital_kind)
            handles, labels = ax1.get_legend_handles_labels()
            ax1.legend(handles, ['$\mu_{}^{{({})}}$'.format(var[-2], var[-1]), capital_kind], loc=0)
        return fig



    def mk_4plots(self, bd, phi):
        # set opacity for plots of micro data:
        micro_alpha=0.2
        # select data for given value of bd and phi
        local_datasets = []
        for d in self.data_sets:
            local_datasets.append(d.xs([bd, phi], level=[0, 1]))

        l_vars = len(self.variable_combos)
        fig = plt.figure(figsize=(12,12))
        axes = [fig.add_subplot(2, 2, i + 1) for i in range(l_vars)]

        legend_locations = [4, 7, 1, 1]
        upper_limits = [0.8, 40, 12, 10]
        lower_limits = [-.6, 0., 0., 0.]

        for j, variables in enumerate(self.variable_combos):
            ax_id = j
            # local data set for specify value of phi
            ldp = local_datasets
            if j == 1:
                print( 'c exception')
                c_ax = axes[ax_id].twinx()
                ldp[0][variables[0]] \
                    .plot(
                    ax=axes[ax_id],
                    color=self.colors[0],
                    alpha=micro_alpha)
                ldp[0][variables[1]] \
                    .plot(
                    ax=c_ax,
                    color=self.colors[1],
                    alpha=micro_alpha)

                c_ax.set_ylim([0, 5])
            else:
                ldp[0][variables] \
                    .plot(
                    ax=axes[ax_id],
                    color=self.colors,
                    alpha=micro_alpha)
            for k, variable in enumerate(variables):
                ax = axes[ax_id]
                if variable == 'g':
                    ax = c_ax

                upper_limit = np.transpose(ldp[0][[variable]].values \
                                           + ldp[1][[variable]].values)[0]
                lower_limit = np.transpose(ldp[0][[variable]].values \
                                           - ldp[1][[variable]].values)[0]
                ax.fill_between(ldp[0].index.values,
                                         upper_limit, lower_limit,
                                         color='k',
                                         alpha=0.05)
                ax.plot(ldp[0].index.values,
                                         upper_limit,
                                         color=self.colors[k],
                                         alpha=0.2)
                ax.plot(ldp[0].index.values,
                                         lower_limit,
                                         color=self.colors[k],
                                         alpha=0.2)
                ldp[2][[variable]] \
                    .plot(ax=ax,
                          color=self.colors[k],
                          legend=False,
                          style='--',
                          linewidth=2
                          )

            ax = axes[ax_id]
            ax.set_ylim([lower_limits[j], upper_limits[j]])
            k = len(variables)
            patches, labels = ax.get_legend_handles_labels()
            labels = self.latex_labels[j]
            if j == 1:
                gArtist = plt.Line2D((0,1),(0,0), color=self.colors[1])
                cArtist = plt.Line2D((0,1),(0,0), color=self.colors[0])
                patches = [cArtist, gArtist]

            lg = ax.legend(patches[:k], labels[:k],
                           loc=legend_locations[ax_id],
                           title='',
                           fontsize=12)
            lg.get_frame().set_alpha(0.8)

            #ax.set_title('')
            #ax.set_loc

        return fig


    def grid_index(self, row, col, n_rows, n_cols):
        """
        calculate index number in subplot grids
        grid_index(0,0) = 1 ~ upper left corner
        to index the axis list, one might use grid_index - 1
        """
        return col + 1 + row * n_cols

In [4]:
import time

bc = 1.25

prt = plot_routines(base_path='/home/jakob/Project_Divestment/PyDivestment/output_data/X6/')



for phi in [round(x, 5) for x in list(np.linspace(0.0, 0.9, 10))]:

    figure = prt.mk_4plots(bc, phi)
    figure.show()
    time.sleep(1)
    figure.savefig('trajectory_bc={}_phi={}.png'.format(bc, phi))
    figure.clf()


c exception


/home/jakob/anaconda3/envs/py36/lib/python3.6/site-packages/matplotlib/figure.py:459: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "


c exception
c exception
c exception
c exception
c exception
c exception
c exception
c exception
c exception


<Figure size 864x864 with 0 Axes>

<Figure size 864x864 with 0 Axes>

...

<Figure size 864x864 with 0 Axes>

<Figure size 864x864 with 0 Axes>

<Figure size 864x864 with 0 Axes>

<Figure size 864x864 with 0 Axes>

<Figure size 864x864 with 0 Axes>

In [35]:
prt.macro_mean.head()

observables               c    g    mu_c^c    mu_c^d    mu_d^c    mu_d^d  \
b_d phi timesteps                                                          
1.0 0.0 0.0        0.500000  4.0  0.250000  0.250000  0.250000  0.250000   
        3.0        1.365525  4.0  0.481582  0.290465  0.290711  0.481344   
        6.0        2.344780  4.0  0.679802  0.399016  0.399588  0.677688   
        9.0        3.416711  4.0  0.882474  0.543989  0.530248  0.864267   
        12.0       4.542702  4.0  1.096102  0.708729  0.661943  1.043784   

observables        time         x         y         z  
b_d phi timesteps                                      
1.0 0.0 0.0         0.0  0.002800  0.004577  0.499471  
        3.0         3.0  0.004939  0.010059  0.449860  
        6.0         6.0  0.032587  0.053850  0.449033  
        9.0         9.0  0.072019  0.092780  0.446535  
        12.0       12.0  0.101931  0.115324  0.444524

In [36]:
prt.micro_mean.head()

observables              Wc        Wd         c    g    mu_c^c    mu_c^d  \
b_d phi timesteps                                                          
1.0 0.0 0.0        0.224685  0.224686  0.500026  4.0  0.250010  0.249999   
        3.0        0.289108  0.285320  1.356732  4.0  0.484207  0.286662   
        6.0        0.348883  0.338601  2.330236  4.0  0.678762  0.388866   
        9.0        0.402206  0.388183  3.373684  4.0  0.881574  0.525347   
        12.0       0.449928  0.433768  4.458839  4.0  1.085731  0.671213   

observables          mu_d^c    mu_d^d       r_c       r_d  time         w  \
b_d phi timesteps                                                           
1.0 0.0 0.0        0.249999  0.250010  0.428196  0.470532   0.0  0.520740   
        3.0        0.285876  0.483039  0.382284  0.363836   3.0  0.673450   
        6.0        0.400636  0.687609  0.335359  0.302892   6.0  0.808962   
        9.0        0.531136  0.881985  0.298121  0.262787   9.0  0.932443   
        12.0       0.673354  1.080633  0.269283  0.234408  12.0  1.045359   

observables               x         y         z  
b_d phi timesteps                                
1.0 0.0 0.0        0.006200  0.005725  0.500261  
        3.0       -0.024732 -0.022156  0.449543  
        6.0        0.000225 -0.000333  0.443455  
        9.0       -0.001380 -0.003810  0.440715  
        12.0       0.025699  0.021278  0.437933

In [44]:
x = np.array(list(range(10)))
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [45]:
2 * x

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])